In [ ]:
minian_path = "."
dpath = "./demo_movies/"

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import warnings
sys.path.append(minian_path)
import itertools as itt
import numpy as np
import xarray as xr
import holoviews as hv
import pandas as pd
from holoviews.operation.datashader import datashade, regrid
from minian.cross_registration import (estimate_shifts, calculate_centroids,
                                       calculate_centroid_distance, calculate_mapping,
                                       group_by_session, resolve_mapping, fill_mapping)
from minian.motion_correction import apply_shifts
from minian.utilities import open_minian, open_minian_mf
from minian.visualization import AlignViewer
hv.notebook_extension('bokeh', width=100)

In [ ]:
minian_df = open_minian_mf(dpath, ['animal', 'session'], result_format='pandas', backend='zarr')

In [ ]:
%%time
shiftds = estimate_shifts(minian_df, template=0)

In [ ]:
%%output size=50
alignviewer = AlignViewer(shiftds)
alignviewer.show()

In [ ]:
shiftds = alignviewer.shiftds

In [ ]:
minian = open_minian_mf(dpath, ['animal', 'session'], result_format='xarray', backend='zarr')

In [ ]:
A_shifted = apply_shifts(minian['A'].chunk(dict(height=-1, width=-1, unit_id='auto')), shiftds['shifts'])

In [ ]:
window = shiftds['temps_shifted'].isnull().sum('session')

In [ ]:
%%output size=50
%%opts Image [height=480, width=752, colorbar=True] (cmap='Viridis')
window, temps_sh = xr.broadcast(window, shiftds['temps_shifted'])
hv_wnd = hv.Dataset(window, kdims=list(window.dims)).to(hv.Image, ['width', 'height'])
hv_temps = hv.Dataset(temps_sh, kdims=list(temps_sh.dims)).to(hv.Image, ['width', 'height'])
regrid(hv_wnd, aggregator='max').relabel("Window") + regrid(hv_temps).relabel("Shifted Templates")

In [ ]:
%%time
cents = calculate_centroids(A_shifted, window==0)

In [ ]:
%%output size=50
%%opts Points [height=480, width=752] {+axiswise +framewise}
cents_hv = hv.Dataset(cents, kdims=['height', 'width', 'unit_id', 'animal', 'session'])
cents_hv.to(hv.Points, kdims=['width', 'height']).overlay('unit_id')

In [ ]:
%%time
dist = calculate_centroid_distance(cents)

In [ ]:
dist_ft = dist[dist['variable', 'distance'] < 5]
dist_ft = group_by_session(dist_ft)

In [ ]:
%%time
mappings = calculate_mapping(dist_ft)
mappings_meta = resolve_mapping(mappings)
mappings_meta_fill = fill_mapping(mappings_meta, cents)

In [ ]:
mappings_meta_fill.to_pickle(os.path.join(dpath, "mappings_meta_fill.pkl"))